In [11]:
import requests
# 测试脚本顶部添加正确的导入
from PIL import Image  # 正确导入Image模块
from io import BytesIO  # 导入字节流处理
import base64          # 导入base64解码


In [12]:
def test_api():
    url = "http://localhost:6006/predict"
    
    # 正确构建文件列表
    files = [
        ("files", ("pair1_left.jpg", open(r"Training_Dataset/0_left.jpg", "rb"), "image/jpeg")),
        ("files", ("pair1_right.jpg", open(r"Training_Dataset/0_right.jpg", "rb"), "image/jpeg")),
        ("files", ("pair2_left.jpg", open(r"Training_Dataset/1_left.jpg", "rb"), "image/jpeg")),
        ("files", ("pair2_right.jpg", open(r"Training_Dataset/1_right.jpg", "rb"), "image/jpeg"))
    ]

    try:
        # 发起请求并设置超时
        response = requests.post(url, files=files, timeout=30)
        response.raise_for_status()

        # 处理成功响应
        if response.status_code == 200:
            for result in response.json():
                print(f"\n文件对: {result['filenames']}")
                print("预测概率：")
                
                # 按概率从高到低排序
                sorted_predictions = sorted(
                    result["predictions"].items(),
                    key=lambda x: x[1],
                    reverse=True
                )
                
                # 打印前3个最可能的诊断结果
                for i, (cls, prob) in enumerate(sorted_predictions[:3]):
                    print(f"TOP{i+1} {cls}: {prob*100:.1f}%")
                
                # 显示拼接图像（仅显示第一个结果）
                if result == response.json()[0]:
                    img_data = result["combined_image"].split(",")[-1]
                    img = Image.open(BytesIO(base64.b64decode(img_data)))
                    img.show()
                
        else:
            print(f"请求失败，状态码: {response.status_code}")
            print(f"错误详情: {response.text}")

    except requests.exceptions.RequestException as e:
        print(f"请求异常: {str(e)}")
        if hasattr(e, 'response') and e.response:
            print(f"响应内容: {e.response.text}")

    finally:
        # 确保关闭所有文件
        for _, file_data in files:
            file_data[1].close()

In [13]:
if __name__ == "__main__":
    test_api()


文件对: ['pair1_left.jpg', 'pair1_right.jpg']
预测概率：
TOP1 N: 61.0%
TOP2 O: 11.5%
TOP3 C: 7.0%



文件对: ['pair2_left.jpg', 'pair2_right.jpg']
预测概率：
TOP1 N: 53.8%
TOP2 H: 27.1%
TOP3 A: 10.5%


In [ ]:
curl -X POST http://localhost:6006/predict \
-F "files=@Training_Dataset/0_left.jpg" \
-F "files=@Training_Dataset/0_right.jpg"

In [1]:
import requests
from PIL import Image
from io import BytesIO
import base64
import os
import time
import matplotlib.pyplot as plt
import numpy as np

def generate_file_pairs(dataset_path, num_pairs=50):
    """自动生成图像对列表"""
    file_pairs = []
    for i in range(num_pairs):
        left_path = os.path.join(dataset_path, f"{i}_left.jpg")
        right_path = os.path.join(dataset_path, f"{i}_right.jpg")
        
        # 验证文件存在性
        if not os.path.exists(left_path) or not os.path.exists(right_path):
            print(f"警告：缺少第 {i} 对图像文件，已跳过")
            continue
            
        file_pairs.extend([
            ("files", (f"pair{i}_left.jpg", open(left_path, "rb"), "image/jpeg")),
            ("files", (f"pair{i}_right.jpg", open(right_path, "rb"), "image/jpeg"))
        ])
    return file_pairs

def display_statistics(results):
    """显示统计摘要"""
    all_probs = []
    class_counts = {cls: 0 for cls in CLASS_NAMES}
    
    for result in results:
        # 记录最高概率类别
        max_cls = max(result["predictions"], key=result["predictions"].get)
        class_counts[max_cls] += 1
        
        # 收集所有概率
        all_probs.extend(result["predictions"].values())

    # 打印统计结果
    print("\n全局统计：")
    print(f"总处理图像对：{len(results)}")
    print(f"平均置信度：{np.mean(all_probs)*100:.1f}%")
    print("\n疾病类型分布：")
    for cls, count in class_counts.items():
        print(f"{cls}: {count} 对")

def visualize_results(results, save_path="diagnosis_report"):
    """可视化所有结果并保存报告"""
    os.makedirs(save_path, exist_ok=True)
    
    plt.figure(figsize=(15, len(results)*3))
    for idx, result in enumerate(results):
        # 解码图像
        img_data = result["combined_image"].split(",")[-1]
        img = Image.open(BytesIO(base64.b64decode(img_data)))
        
        # 保存原始图像
        img.save(os.path.join(save_path, f"pair_{idx}.jpg"))
        
        # 创建可视化面板
        plt.subplot(len(results), 2, idx*2+1)
        plt.imshow(img)
        plt.axis('off')
        
        # 添加预测信息
        plt.subplot(len(results), 2, idx*2+2)
        sorted_pred = sorted(result["predictions"].items(), key=lambda x: x[1], reverse=True)
        classes = [x[0] for x in sorted_pred[:3]]
        probs = [x[1] for x in sorted_pred[:3]]
        plt.barh(classes, probs)
        plt.xlim(0, 1)
        plt.title(f"Pair {idx} 诊断结果")
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "summary_report.png"))
    plt.close()

def test_api_large_scale():
    url = "http://localhost:6006/predict"
    dataset_path = "Training_Dataset"
    
    try:
        # 生成50对图像文件（自动跳过缺失文件）
        files = generate_file_pairs(dataset_path, 50)
        if not files:
            raise ValueError("未找到有效的图像文件对")
            
        # 添加性能监控头
        headers = {"X-Performance-Monitor": "true"}
        
        # 分批处理（每批10对）
        batch_size = 20  # 根据服务端MAX_BATCH_SIZE调整
        all_results = []
        
        start_time = time.time()
        
        for i in range(0, len(files), batch_size*2):
            batch_files = files[i:i+batch_size*2]
            
            # 发送请求
            response = requests.post(url, files=batch_files, headers=headers, timeout=120)
            response.raise_for_status()
            
            if response.status_code == 200:
                batch_results = response.json()
                all_results.extend(batch_results)
                
                # 打印进度
                print(f"已处理 {min(i//2 + batch_size, 50)}/{50} 对图像")
            else:
                print(f"批次 {i//2//batch_size +1} 处理失败，状态码：{response.status_code}")
                
        total_time = time.time() - start_time
        print(f"\n总耗时：{total_time:.2f}秒")
        print(f"平均每对处理时间：{total_time/len(all_results):.2f}秒")

        # 显示统计摘要
        display_statistics(all_results)
        
        # 生成可视化报告
        visualize_results(all_results)
        
        # 在Notebook中显示汇总图像
        if 'IPython' in globals():
            from IPython.display import Image as IPImage
            display(IPImage(filename="diagnosis_report/summary_report.png"))

    except Exception as e:
        print(f"发生严重错误：{str(e)}")
        
    finally:
        # 确保关闭所有文件
        for _, file_data in files:
            file_data[1].close()

if __name__ == "__main__":
    # 配置全局参数
    CLASS_NAMES = ['N','D','G','C','A','H','M','O']  # 与服务端一致
    
    # 执行测试
    test_api_large_scale()

已处理 20/50 对图像
已处理 40/50 对图像
已处理 50/50 对图像

总耗时：10.30秒
平均每对处理时间：0.21秒

全局统计：
总处理图像对：50
平均置信度：20.6%

疾病类型分布：
N: 7 对
D: 10 对
G: 1 对
C: 1 对
A: 1 对
H: 4 对
M: 4 对
O: 22 对


/tmp/ipykernel_1219/1730167142.py:76: UserWarning: Glyph 35786 (\N{CJK UNIFIED IDEOGRAPH-8BCA}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_1219/1730167142.py:76: UserWarning: Glyph 26029 (\N{CJK UNIFIED IDEOGRAPH-65AD}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_1219/1730167142.py:76: UserWarning: Glyph 32467 (\N{CJK UNIFIED IDEOGRAPH-7ED3}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_1219/1730167142.py:76: UserWarning: Glyph 26524 (\N{CJK UNIFIED IDEOGRAPH-679C}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_1219/1730167142.py:77: UserWarning: Glyph 35786 (\N{CJK UNIFIED IDEOGRAPH-8BCA}) missing from current font.
  plt.savefig(os.path.join(save_path, "summary_report.png"))
/tmp/ipykernel_1219/1730167142.py:77: UserWarning: Glyph 26029 (\N{CJK UNIFIED IDEOGRAPH-65AD}) missing from current font.
  plt.savefig(os.path.join(save_path, "summary_report.png"))
/tmp/ipykernel_1219/1730167142.py:77: UserWarning: Glyph

main_will

In [2]:
import requests
from PIL import Image
from io import BytesIO
import base64
import os
import time
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# 配置参数
CLASS_NAMES = ['N','D','G','C','A','H','M','O']  # 与服务端完全一致
SERVER_URL = "http://localhost:6006/predict"
DATASET_PATH = "Training_Dataset"
REPORT_DIR = "diagnosis_reports"
BATCH_SIZE = 10  # 与服务端MAX_BATCH_SIZE匹配

def generate_file_pairs(num_pairs=50):
    """智能生成有效图像对列表"""
    valid_pairs = []
    for i in range(num_pairs):
        left_path = os.path.join(DATASET_PATH, f"{i}_left.jpg")
        right_path = os.path.join(DATASET_PATH, f"{i}_right.jpg")
        
        if os.path.exists(left_path) and os.path.exists(right_path):
            valid_pairs.append((left_path, right_path))
        else:
            print(f"跳过不完整对: Pair {i}")
    
    if not valid_pairs:
        raise ValueError("未找到任何有效图像对")
    return valid_pairs

def analyze_results(results):
    """深入分析预测结果"""
    stats = {
        'class_dist': {cls: 0 for cls in CLASS_NAMES},
        'confidence_ranges': {cls: [] for cls in CLASS_NAMES},
        'response_times': []
    }
    
    for result in results:
        # 最高置信度类别
        max_cls = max(result['predictions'], key=result['predictions'].get)
        stats['class_dist'][max_cls] += 1
        
        # 收集各分类置信度
        for cls in CLASS_NAMES:
            stats['confidence_ranges'][cls].append(result['predictions'][cls])
            
        # 响应时间分析
        stats['response_times'].append(result['processing_time'])
    
    # 生成统计摘要
    print("\n=== 全局统计 ===")
    print(f"总处理图像对: {len(results)}")
    print(f"平均响应时间: {np.mean(stats['response_times']):.2f}s ± {np.std(stats['response_times']):.2f}")
    
    print("\n=== 疾病分布 ===")
    for cls, count in stats['class_dist'].items():
        print(f"{cls}: {count} 对 (占比 {count/len(results):.1%})")
    
    print("\n=== 置信度分析 ===")
    for cls in CLASS_NAMES:
        confs = stats['confidence_ranges'][cls]
        print(f"{cls}: 平均 {np.mean(confs):.2f} 范围 [{np.min(confs):.2f}, {np.max(confs):.2f}]")

def generate_visual_report(results):
    """生成可视化诊断报告"""
    os.makedirs(REPORT_DIR, exist_ok=True)
    
    # 创建汇总图
    plt.figure(figsize=(20, 6 * len(results)))
    for idx, result in enumerate(results):
        # 解码图像
        img_data = result['combined_image'].split(",")[-1]
        img = Image.open(BytesIO(base64.b64decode(img_data)))
        
        # 保存原始图像
        img.save(os.path.join(REPORT_DIR, f"pair_{idx}_combined.jpg"))
        
        # 绘制图像和图表
        plot_position = 2 * idx + 1
        plt.subplot(len(results), 2, plot_position)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Pair {idx} - {result['filenames'][0]}")
        
        # 绘制置信度条形图
        plt.subplot(len(results), 2, plot_position + 1)
        sorted_pred = sorted(result['predictions'].items(), key=lambda x: x[1], reverse=True)[:5]
        classes = [x[0] for x in sorted_pred]
        probs = [x[1] for x in sorted_pred]
        plt.barh(classes, probs, color='skyblue')
        plt.xlim(0, 1)
        plt.title(f"Top 5 诊断置信度\n最大: {max(probs):.2f} 平均: {np.mean(probs):.2f}")
    
    plt.tight_layout()
    plt.savefig(os.path.join(REPORT_DIR, "diagnosis_summary.png"))
    plt.close()

def batch_predict(pairs):
    """执行分批预测"""
    all_results = []
    
    with tqdm(total=len(pairs), desc="处理进度") as pbar:
        for i in range(0, len(pairs), BATCH_SIZE):
            batch_pairs = pairs[i:i+BATCH_SIZE]
            
            # 构建多部分请求
            files = []
            for left_path, right_path in batch_pairs:
                files.append(('files', (os.path.basename(left_path), open(left_path, 'rb'), 'image/jpeg')))
                files.append(('files', (os.path.basename(right_path), open(right_path, 'rb'), 'image/jpeg')))
            
            # 发送请求
            start_time = time.time()
            response = requests.post(SERVER_URL, files=files)
            response.raise_for_status()
            
            # 添加处理时间
            batch_results = response.json()
            for res in batch_results:
                res['processing_time'] = time.time() - start_time
            
            all_results.extend(batch_results)
            pbar.update(len(batch_pairs))
            
            # 及时关闭文件
            for _, file_tuple in files:
                file_tuple[1].close()
    
    return all_results

def main():
    try:
        print("正在准备测试数据...")
        valid_pairs = generate_file_pairs(50)
        
        print("开始批量预测...")
        results = batch_predict(valid_pairs)
        
        print("\n生成分析报告...")
        analyze_results(results)
        generate_visual_report(results)
        
        print(f"\n报告已保存至: {os.path.abspath(REPORT_DIR)}")
        
    except requests.exceptions.RequestException as e:
        print(f"请求失败: {str(e)}")
        if e.response is not None:
            print(f"错误详情: {e.response.text}")
    except Exception as e:
        print(f"发生错误: {str(e)}")
    finally:
        # 清理打开的文件
        pass  # 已确保在batch_predict中关闭

if __name__ == "__main__":
    main()

正在准备测试数据...
开始批量预测...


处理进度: 100%|██████████| 50/50 [00:09<00:00,  5.34it/s]



生成分析报告...

=== 全局统计 ===
总处理图像对: 50
平均响应时间: 1.87s ± 2.50

=== 疾病分布 ===
N: 7 对 (占比 14.0%)
D: 10 对 (占比 20.0%)
G: 1 对 (占比 2.0%)
C: 1 对 (占比 2.0%)
A: 1 对 (占比 2.0%)
H: 5 对 (占比 10.0%)
M: 4 对 (占比 8.0%)
O: 21 对 (占比 42.0%)

=== 置信度分析 ===
N: 平均 0.23 范围 [0.00, 0.77]
D: 平均 0.41 范围 [0.00, 0.99]
G: 平均 0.07 范围 [0.00, 0.57]
C: 平均 0.04 范围 [0.00, 0.93]
A: 平均 0.06 范围 [0.00, 0.84]
H: 平均 0.20 范围 [0.00, 1.00]
M: 平均 0.11 范围 [0.00, 0.99]
O: 平均 0.54 范围 [0.01, 1.00]


/tmp/ipykernel_1285/1956463071.py:98: UserWarning: Glyph 35786 (\N{CJK UNIFIED IDEOGRAPH-8BCA}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_1285/1956463071.py:98: UserWarning: Glyph 26029 (\N{CJK UNIFIED IDEOGRAPH-65AD}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_1285/1956463071.py:98: UserWarning: Glyph 32622 (\N{CJK UNIFIED IDEOGRAPH-7F6E}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_1285/1956463071.py:98: UserWarning: Glyph 20449 (\N{CJK UNIFIED IDEOGRAPH-4FE1}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_1285/1956463071.py:98: UserWarning: Glyph 24230 (\N{CJK UNIFIED IDEOGRAPH-5EA6}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_1285/1956463071.py:98: UserWarning: Glyph 26368 (\N{CJK UNIFIED IDEOGRAPH-6700}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_1285/1956463071.py:98: UserWarning: Glyph 22823 (\N{CJK UNIFIED IDEOGRAPH-5927}) missing from current font.
  plt.tight_l


报告已保存至: /root/diagnosis_reports
